# Experiment and path specification

In [ ]:
# Specify the experiment for analysis
# experiment='cxic00318'
# runNumber = 123
experiment='cxix40218'
# runNumber=31 # includes cspad info
# runNumber=32 # includes cspad info
runNumber=33 # includes cspad info
runNumber=29 # includes cspad info
runNumber=28 # includes cspad info
runNumber=27 # includes cspad info
runNumber=26 # includes cspad info
runNumber=14 # includes cspad info
runNumber=13 # includes cspad info
runNumber=21 # includes cspad info
runNumber=37 # includes cspad info
runNumber=39 # includes cspad info
runNumber=49 # includes cspad info
runNumber=50 # includes cspad info


# Set the installation and output path
import os
os.environ['INSTALLPATH']= '/cds/home/i/igabalsk/TRXS-Run18'
os.environ['OUTPUTPATH']= '/cds/data/psdm/%s/%s/scratch' % (experiment[0:3],experiment)

# Import Libraries

In [ ]:
# Magic iPython command to enable plotting
%matplotlib inline

# Load in the pythonBatchMagic library
import sys
sys.path.insert(0, os.environ['INSTALLPATH']+'/Libraries/pythonBatchMagic')
from pythonBatchMagic import *

# Determine current user
currentUser, error = unixCMD("echo $USER")
currentUser = currentUser.strip()
print(currentUser)
os.environ['RESULTSPATH']= ('/cds/data/psdm/%s/%s/results/%s' % (experiment[0:3],experiment,currentUser)).strip()
if not os.path.exists(os.environ['RESULTSPATH']):
    os.mkdir(os.environ['RESULTSPATH'])

# Leveraging the batch queue to quickly grab point data

## Loading libraries

In [ ]:
sys.path.insert(0, os.environ['INSTALLPATH']+'/Libraries/LCLS')
from LCLSdefault import *

sys.path.insert(0, os.environ['INSTALLPATH']+'/Libraries/mattsLibraries')
from picklez import *

# Load in the get data library
from dataAnalysis import *

# Load in the batch library for lcls
from lclsBatch import *

## Grabbing point data across nodes

In [ ]:
detDict = {
    'uvint':{'name': 'Acqiris','get-function':'getAcqirisSum2'},
           'uvint1':{'name': 'Acqiris','get-function':'getAcqirisSum1'},
           'xrayEnergy':{'name':'FEEGasDetEnergy','get-function': 'getXrayEnergy' },
           'ebeamcharge':{'name':'EBeam', 'get-function': 'getEbeamCharge'},
          'stageencoder':{'name':'CXI:LAS:MMN:04.RBV','get-function':'getStageEncoder'},
          'ttfltpos':{'name':'CXI:TIMETOOL:FLTPOS', 'get-function': 'getTTFltPos'},
          'vitpos':{'name':'LAS:FS5:VIT:FS_TGT_TIME_DIAL', 'get-function': 'getTTFltPos'},
           'xint':{'name':'CxiDg3_Imb01','get-function': 'getIPM' },
           'xpos':{'name':'CxiDg3_Imb01','get-function': 'getXPos' },
           'ypos':{'name':'CxiDg3_Imb01','get-function': 'getYPos' },
          'seconds':{'name':'seconds', 'get-function': 'getSeconds'},
          'nanoseconds':{'name':'nanonseconds', 'get-function': 'getNanoseconds'},
          'fiducials':{'name':'fiducials', 'get-function': 'getFiducials'},
          'roi1':{'name':'jungfrau4M', 'get-function': 'getCSPADsum1'},
          'roi2':{'name':'jungfrau4M', 'get-function': 'getCSPADsum2'},
          'roi3':{'name':'jungfrau4M', 'get-function': 'getCSPADsum3'},
          'roi4':{'name':'jungfrau4M', 'get-function': 'getCSPADsum4'}
          }



NSHOTS = 100000
ngrab = 1000
rank = int( float(NSHOTS)/float(ngrab) )
print('Grabbing %d shots, %d events at a time, on %d nodes' %(NSHOTS,ngrab,rank))
batchThreads = batchPointDataGrabber( detDict, eventMax=int(rank*ngrab), experiment=experiment, runNumber=runNumber, rank=int(rank) )
# batchThreads.Queue = 'psanaq'
batchThreads.Queue = 'psfehhiprioq'
batchThreads.RunType = 'python2'
batchThreads.start()

## Check job status

In [ ]:
stdoutdata, stderrdata = bjobs()
print stdoutdata
print stderrdata


## Kill threads if they die

In [ ]:
batchThreads.requestStop()
# time.sleep(5)
stdoutdata, stderrdata = bkill(killAll=True)
print stdoutdata
print stderrdata

In [ ]:
batchThreads.status

In [ ]:
batchThreads.gather()

## Drop empty array values

In [ ]:
batchThreads.detArrays['uvint'].shape

In [ ]:
pointData = batchThreads.detArrays
# print pointData

def dropEmpty(pointData):

    filledBin = (pointData['seconds'] > 0)

    for key in pointData.keys():
        pointData[key]=pointData[key][filledBin] 
        
    return pointData

def dropNaN(pointData):
    goodBin = np.ones(pointData['seconds'].shape, dtype=bool)
    for key in pointData.keys():
        goodBin = goodBin & ~np.isnan(pointData[key])
    for key in pointData.keys():
        pointData[key]=pointData[key][goodBin]
    return pointData

print len(pointData['seconds'])
pointData_masked = dropNaN(dropEmpty(pointData))
print 'Number of good shots: ', len(pointData_masked['seconds'])

## Save point data

In [ ]:
save_obj( pointData, os.environ['RESULTSPATH']+'/pointData-run-%d' % runNumber )

In [ ]:
# pointData = load_obj(os.environ['RESULTSPATH']+'/pointData-run-%d' % 38)

## Plot histogram of result

In [ ]:
detArrays = pointData

fig2, ax2 = plt.subplots(1, 1)
ax2.hist(detArrays['xrayEnergy'], bins=100,normed=False, alpha=0.5);

In [ ]:
detArrays = pointData

fig2, ax2 = plt.subplots(1, 1)
ax2.hist(detArrays['stageencoder'], bins=200,normed=False, alpha=0.5);
plt.xlabel(' CXI:LAS:MMN:04.RBV ')
plt.ylabel(' counts per bin ')

In [ ]:
plt.scatter(detArrays['uvint'],detArrays['xint'])

In [ ]:
np.unique(detArrays['vitpos'])

In [ ]:
plt.scatter(detArrays['roi2'],detArrays['roi1'])
plt.xlabel('Jungfrau sum 2')
plt.ylabel('Jungfrau sum 1')

In [ ]:
goodIdx = detArrays['roi2'] > 1e4
plt.scatter(detArrays['uvint'][goodIdx],detArrays['xint'][goodIdx])
plt.xlabel('UV intensity')
plt.ylabel('Photodiode / Gas det (arb)')


In [ ]:
pos = detArrays['stageencoder'][goodIdx] 
roi1 = detArrays['roi1'][goodIdx]
roi2 = detArrays['roi2'][goodIdx]
roi3 = detArrays['roi3'][goodIdx]
roi4 = detArrays['roi4'][goodIdx]
# roi5 = detArrays['roi5'][goodIdx]
# roi6 = detArrays['roi6'][goodIdx]
# roi7 = detArrays['roi7'][goodIdx]
# roi0 = detArrays['roi0'][goodIdx]
np.unique(pos)

In [ ]:
nb=5
counts,edges = np.histogram( pos,bins=np.unique(pos))
bins,edges = np.histogram( pos,bins=np.unique(pos), weights=(roi2-roi3)/(roi1+roi2+roi3))

In [ ]:
plt.plot(edges[1:] , bins/counts )
plt.xlabel(' CXI:LAS:MMN:04.RBV ')
plt.ylabel(' roi1/roi2 ')
# plt.savefig('run-13-t0.png')

In [ ]:
goodVitPos = detArrays['vitpos'][goodIdx] 
uvB = detArrays['uvint'][goodIdx]
uvA = detArrays['uvint1'][goodIdx]
xint = detArrays['xrayEnergy'][goodIdx]

In [ ]:
import scipy.stats
diodeB = []
diodeA = []
xInt = []
centers = []
rs=[]
for idx, b in enumerate(bins):
    goodIdxs = (goodVitPos >= edges[idx]) & (goodVitPos < edges[idx+1])
    diodeA = uvA[goodIdxs]
    diodeB = uvB[goodIdxs]
    xInt = xint[goodIdxs]
    x= xInt
    y=diodeB #/diodeA
    try:
        slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)
        centers.append( float(edges[idx+1] + edges[idx])/2.)
        rs.append(r_value)
    except ValueError:
        continue
        
print(centers)

In [ ]:
plt.plot(np.array(centers)*1e3,rs)
plt.xlabel('vitara (ns)')
plt.ylabel('R - Correlation coefficient')
plt.title('Correlation coefficient for each delay position')
# plt.savefig('correlation-coefficient-v-vitara-delay-run13.png')

In [ ]:

diodeA

In [ ]:
import scipy.stats
x= xInt
y=diodeB/diodeA
slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)

print("slope: %f    intercept: %f" % (slope, intercept))
print(r_value)

plt.scatter(xInt,diodeB/diodeA)
plt.plot(xInt, slope*x + intercept)
plt.ylabel('diodeB/diodeA')
plt.xlabel('xray energy at gas det')
plt.title('Best fit at 54 ps vitara delay')
plt.savefig('vitara-fit-54ps-run13.png')

In [ ]:
plt.hist(goodVitPos,100)

# Quick fixes

In [ ]:
detArrays0 = load_obj( BATCHDIR + '/Output/nodePointDataGrabber%d-run-%d' % (0, runNumber) )

In [ ]:
detArrays0['uvint'].shape

In [ ]:
startNode = 0
rank = 20
detArrays = None
for node in range(0,rank):
    try:
        detArrays = load_obj( BATCHDIR + '/Output/nodePointDataGrabber%d-run-%d' % (node,runNumber) )
        startNode = node
        break
    except IOError:
        print('Could not read in node %d' % node)
for node in range(startNode+1,rank):
    try:
        detArrays0 = load_obj( BATCHDIR + '/Output/nodePointDataGrabber%d-run-%d' % (node, runNumber) )
        for key in detArrays.keys():
            detArrays[key] = np.append(detArrays[key], detArrays0[key])
#                     self.detArrays[key] += detArrays0[key]
    except IOError:
        print('Could not read in node %d' % node)

In [ ]:
detArrays['uvint'].shape

In [ ]:
detArrays.keys()